In [ ]:
from scipy.spatial import cKDTree

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
output_notebook()

from neuprint import Client, attach_synapses_to_skeleton, fetch_neurons, fetch_skeleton, NeuronCriteria as NC, fetch_synapses, SynapseCriteria as SC, skeleton_segments

Loading BokehJS ...

In [2]:
c = Client('neuprint.janelia.org', 'hemibrain:v1.2.1')

In [3]:
#neurons, syn_counts = fetch_neurons(NC(status='Traced', cropped=False))

### Fetch the skeleton and synapses for a body

In [4]:
body = 1136399017
synapses = fetch_synapses(body)
skeleton = fetch_skeleton(body, heal=True)

# Attach synapses to the skeleton at their nearest skeleton node
augmented_skeleton = attach_synapses_to_skeleton(skeleton, synapses)

### Plot the new skeleton

In [5]:
# Create a table of line segments for plotting
segments = skeleton_segments(augmented_skeleton)

colors = {
    'neurite': 'green',
    'pre': 'blue',
    'post': 'red'
}
segments['color'] = segments['structure'].map(colors)

In [6]:
segments.head()

,rowId,x,y,z,radius,link,structure,rowId_parent,link_parent,x_parent,y_parent,z_parent,radius_parent,length,avg_radius,volume,color
0,2,12746.700195,30370.699219,15788.700195,55.4641,1,neurite,1,-1,12798.000000,30268.000000,15812.000000,21.0000,117.139577,38.23205,574334.406286,green
1,3,12727.200195,30411.199219,15767.599609,68.0812,2,neurite,2,1,12746.700195,30370.699219,15788.700195,55.4641,49.656165,61.77265,597341.554444,green
2,1156,12812.000000,30436.000000,15868.000000,12.7990,2,neurite,2,1,12746.700195,30370.699219,15788.700195,55.4641,121.723932,34.13155,503498.199869,green
3,1753,12714.000000,30360.000000,15783.000000,0.0000,2,pre,2,1,12746.700195,30370.699219,15788.700195,55.4641,34.875038,27.73205,112348.491936,blue
4,1795,12775.000000,30358.000000,15756.000000,0.0000,2,post,2,1,12746.700195,30370.699219,15788.700195,55.4641,45.071630,27.73205,145196.392390,red


In [7]:
p = figure()
p.y_range.flipped = True

# Plot skeleton segments (in 2D)
p.segment(x0='x', x1='x_parent',
          y0='z', y1='z_parent',
          line_color='color',
          line_width=2,
          source=segments)

# Plot synapses as points, too.
syn_segments = segments.query('structure != "neurite"')
p.scatter(syn_segments['x'], syn_segments['z'], color=syn_segments['color'], width=5)

show(p)